# Formato para la implementación de los ejercicios de la Tarea  4 en Python

### Los programas implementados para la  solución de los ejercicios se evaluarán ejecutando una notebook con el formato que se describe a continuación. Cada estudiante debe asegurarse que sus programas se ejecutan de manera correcta y en un tiempo razonable utilizando este formato. 

### Los programas se pueden implementar en Python ó R. No deben implementarse en los dos lenguajes pues solamente se evaluará en uno de ellos. El código en Python o R podrá incluirse directamente en las celdas de la notebook o implementarse independientemente para ser invocado DESDE LA NOTEBOOK. Es decir, los programas no se evaluarán independientemente sino a partir del llamado que se hace en la notebook. 

In [1]:
# Ejemplo de resolución del problema QAP en Python

# Importamos las librerías imprescindibles para la ejecución del ejercicio
import numpy as np


def ReadQAPInstance(fname):
# Este programa recibe el nombre y ubicación de un fichero con los datos de una instancia del problema QAP
    # y devuelve una estructura con los datos de la instancia.
    hdl = open(fname, 'r')
    mylist = hdl.readlines()
    hdl.close()
    n = eval(mylist[0])

    w  = np.zeros((n,n))
    for i in range(n):
        for j,val in enumerate(mylist[i+1].split()):     
            w[i,j]=eval(val)
    
    d  = np.zeros((n,n))
    for i in range(n,2*n):
        for j,val in enumerate(mylist[i+1].split()):     
            d[i-n,j]=eval(val)
    
    QAPInstance = { "weights" : w,
                   "distances" : d}

    return QAPInstance

In [2]:
def QAPEvaluator( Dist_Matrix, Weigth_Matrix, permutation):
    n = Dist_Matrix.shape[0]   
    perm = np.asarray(permutation) - 1                           # La representación en python comienza en cero   
    val_qap = 0  # 
    for i in range(n):
        for j in range(n):    
            val_qap = val_qap + Weigth_Matrix[i,j] * Dist_Matrix[perm[i],perm[j]]# Coste utlizacion entre instalaciones consecutivas   
  
    # Finalmente se devuelve el resultado
    return val_qap  

In [3]:
# Swap crea una vecindad basada en el operador de intercambio entre posiciones
# Todas las permutaciones que se pueden obtener como un swap entre la primera posición
# y cualquiera de las restantes están en la vecindad
def getAllPairwiseComps(perm):
    n = perm.shape[0]
    n_neighbors = int( n*(n-1)/2 )           # Número de vecinos
    neighbors = np.zeros((n_neighbors,n),dtype=int) # Guardaremos todos los vecinos en neighbors 
    ind = 0
    for j in range(n):
        for i in range(j+1,n):
            neighbors[ind,:] = perm 
            neighbors[ind,i] = perm[j]
            neighbors[ind,j] = perm[i]  
            ind = ind + 1   
    return neighbors

In [4]:
getAllPairwiseComps(np.asarray([1,2,3,4]))

array([[2, 1, 3, 4],
       [3, 2, 1, 4],
       [4, 2, 3, 1],
       [1, 3, 2, 4],
       [1, 4, 3, 2],
       [1, 2, 4, 3]])

In [5]:
def QAPAdvLocalSearch(fname,permutation,maxevals,k):    
    
    best_vals = []
    init_sol = np.asarray( permutation )
    
    my_QAP_Instance = ReadQAPInstance(fname)  
    
    Dist_Matrix       = my_QAP_Instance["distances"]
    Weigth_Matrix     = my_QAP_Instance["weights"]
    
    multiStart = 100
    n = Dist_Matrix.shape[0] 
    best_val = np.zeros((multiStart,1))
    best_sol = np.zeros((multiStart,n))
    number_evaluations = 0
    ind = 0
    aux_perm = np.zeros((multiStart,n)).astype( int )  
    flag1 = True
    while (ind <= multiStart-1) and (flag1==True):
   
        flag2 = False
        while ( flag2 == False ):
            tmp = np.random.permutation(n)   
            flag2 = True
            for ix in range(aux_perm.shape[0]):
                if all(tmp == aux_perm[ix,:]):
                    flag2 = False
            
        aux_perm[ind,:] = tmp    # Se genera una solución aleatoria factible
        sol =  init_sol[aux_perm[ind,:]]
                
        # En los siguientes pasos se implementa la evaluación la búsqueda local en Python   
        n = Dist_Matrix.shape[1]   
        best_val[ind] = QAPEvaluator( Dist_Matrix, Weigth_Matrix, sol )              # Mejor valor
        best_sol[ind,:] = sol                                             # Mejor solución 
        improvement = True
        while improvement:                    # Mientras se mejore el valor de la función
            neighbors = getAllPairwiseComps(best_sol[ind,:])       # Todos los vecinos
            n_neighbors = neighbors.shape[0]
            best_val_among_neighbors = int(best_val[ind])
            for i in range(n_neighbors):                    # Se recorren todos los vecinos buscando el mejor 
                sol = neighbors[i,:]  
                fval =  QAPEvaluator( Dist_Matrix, Weigth_Matrix, sol )    # Se evalua la función
                if fval<best_val_among_neighbors:             # Si es mejor que el mejor valor entre los vecinos hasta el momento
                    best_val_among_neighbors = fval             # se actualiza el mejor valor
                    best_sol_among_neighbors = sol  
                number_evaluations =  number_evaluations + 1  # Se calcula es número de evaluaciones
                if np.mod(number_evaluations,k)==0:
                    best_vals.append( best_val_among_neighbors )
                improvement = (best_val_among_neighbors<best_val[ind]) #  Se determina si ha habido mejora con respecto al ciclo anterior
            if improvement:                                
                best_val[ind] = best_val_among_neighbors           # Se actualiza el mejor valor y la mejor solución 
                best_sol[ind,:] = best_sol_among_neighbors      
                #print(best_val,best_sol, number_evaluations)
            if (number_evaluations > maxevals):
                flag1 = False
                break
        
        ind = ind+1;
    
    best_val = best_val[0:ind]
    selIx = np.where(best_val == min(best_val))[0]
    selIx = selIx[0]
    best_sol = best_sol[selIx,:].astype(int)

    return best_vals, best_sol


In [6]:
# En esta parte comprobamos la implementación de los programas. 
# Esta celda no debe ser modificada


maxevals=1000
k=20


# Evaluación del algoritmo para 5 instancias de tamaño 10
filename_base = '../Instances/QAP/Cebe.qap.n10.'
mypermutation_10 = [1,3,2,4,10,9,8,7,6,5]
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = QAPAdvLocalSearch(fname,mypermutation_10,maxevals,k)  
  print(10,i+1,best_val,best_sol)  
 

# Evaluación del algoritmo para 5 instancias de tamaño 20
filename_base = '../Instances/QAP/Cebe.qap.n20.'
mypermutation_20 = range(1,21)
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = QAPAdvLocalSearch(fname,mypermutation_20,maxevals,k)  
  print(20,i+1,best_val,best_sol)  


# Evaluación del algoritmo para 5 instancias de tamaño 30
filename_base = '../Instances/QAP/Cebe.qap.n30.'
mypermutation_30 = range(1,31)
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = QAPAdvLocalSearch(fname,mypermutation_30,maxevals,k)  
  print(30,i+1,best_val,best_sol)  



    

10 1 [10603220.0, 10048492.0, 9187536.0, 9187536.0, 7234987.0, 7234987.0, 7234987, 6650388.0, 6650388.0, 6650388, 6422142.0, 6422142, 6413164.0, 6413164, 6333541.0, 6333541, 6333541, 6333541, 11559931.0, 11034416.0, 10130573.0, 7379769.0, 6719002.0, 6719002.0, 6719002, 6719002, 6719002, 11676701.0, 10229100.0, 8191336.0, 8191336.0, 7218116.0, 7119555.0, 7119555, 6155800.0, 6155800.0, 6096748.0, 5553467.0, 5484955.0, 5484955.0, 5484955, 5302938.0, 5302938, 5302938, 5302938, 12175596.0, 12175596.0, 9394431.0, 8564107.0, 7634451.0, 7634451.0] [ 1  3 10  9  4  7  6  2  5  8]
10 2 [217029.0, 214242.0, 208367.0, 208367.0, 202889.0, 202889.0, 202807, 200537.0, 200537.0, 199153.0, 199153.0, 199153, 199153, 223122.0, 210281.0, 210281, 206655.0, 206655.0, 205440.0, 205440.0, 205440, 202504.0, 202504, 202504, 232193.0, 230007.0, 229835.0, 224634.0, 215221.0, 204962.0, 204962.0, 200226.0, 199821.0, 191405.0, 191405.0, 191405.0, 191405, 191405, 213291.0, 213291.0, 204613.0, 204613.0, 204613, 199784

30 4 [2138948.0, 2129445.0, 2129445.0, 2129445.0, 2129445.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2128482.0, 2127214.0, 2113519.0, 2113519.0, 2111948.0, 2111539.0, 2111539.0, 2110884.0, 2110884.0, 2110884.0, 2110884.0, 2110884.0, 2110884.0, 2110884.0, 2110884.0, 2105265.0, 2105265.0, 2105265.0, 2105265.0, 2105265.0, 2105265.0, 2104193.0, 2104193.0, 2102995.0, 2090990.0, 2090990.0, 2089231.0, 2086906.0, 2086906.0, 2086322.0, 2086322.0, 2086322.0, 2086322.0, 2086322.0, 2086322.0, 2086322.0, 2081262.0, 2081262.0, 2081262.0, 2081262.0, 2081262.0, 2081262.0, 2081262.0, 2081262.0, 2078134.0] [19 18  2  3 27 24  6 21  1 30 23 29 28 10 12 26  8 20 22  4  5 17 25 14
  7 16 13 11  9 15]
30 5 [145999318.0, 145999318.0, 145999318.0, 145999318.0, 145999318.0, 145999318.0, 145999318.0, 145999318.0, 145999318.0, 145999318.0, 145589649.0, 145589649.0, 145589649.0, 145589649.0

In [7]:
def QAPEvaluator( Dist_Matrix, Weigth_Matrix, permutation):
    n = Dist_Matrix.shape[0]   
    perm = np.asarray(permutation)# La representación en python comienza en cero   
    val_qap = 0  # 
    for i in range(n):
        for j in range(n):    
            val_qap = val_qap + Weigth_Matrix[i,j] * Dist_Matrix[perm[i],perm[j]]# Coste utlizacion entre instalaciones consecutivas   
  
    # Finalmente se devuelve el resultado
    return (val_qap,)

In [8]:
import random
from deap import algorithms, base, creator, tools

# Ejemplo de la implementación del algoritmo evolutivo avanzado para el problema QAP en Python
def QAPAdvEA(fname,psize,ngen):
    evals = np.zeros((ngen))
    best_sol = []
 
    my_QAP_Instance = ReadQAPInstance(fname)  # Se lee la instancia
    
    Dist_Matrix       = my_QAP_Instance["distances"]
    Weigth_Matrix     = my_QAP_Instance["weights"]
    
    # En los siguientes pasos se implementa el algoritmo evolutivo para el problema QAP
    # Se crea una clase FitnessMax para la maximización de funciones
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    # Se crea una clase individuo asociada a la clase FitnessMin
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    # Heredamos las clases y funciones implementadas como parte de DEAP
    toolbox = base.Toolbox()
    
    # Utilizaremos una representación int aleatoria entre 1 y 10
    IND_SIZE = int(fname[-4:-2])
    toolbox.register("indices", random.sample,range(0,IND_SIZE,1),IND_SIZE)
    
    # Definimos que nuestros individuos tendrán secuencias de tamaño IND_SIZE  
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    
    # Definimos la población a partir de los individuos
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Asociamos como función de aptitud la función QAPEvaluator
    toolbox.register("evaluate", QAPEvaluator,Dist_Matrix,Weigth_Matrix)
    
    # Nuestro operador de cruzamiento basado en correspondencia parcial
    toolbox.register("mate", tools.cxPartialyMatched)
    
    # El operador de mutación con una probabilidad de mutación de 0.05
    toolbox.register("mutate", tools.mutShuffleIndexes,indpb=0.05)
    
    # Usaremos selección por mejor individuo
    toolbox.register("select", tools.selTournament, tournsize=10)


    # La población tendrá 150 individuos
    pop = toolbox.population(n=psize)
    # El algoritmo evolutivo simple utiliza los siguientes parámetros
    # Probabilidad de cruzamiento 0.5
    # Probabilidad de aplicar el operador de mutación 0.2
    # Número de generaciones ngen
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)

    pop, logbook = algorithms.eaSimple(pop, toolbox, stats=stats,cxpb=0.5, mutpb=0.05, ngen=ngen, verbose=False)
    evals = logbook.select('min')[1:ngen+1]
    best_sol = list( ( np.asarray( tools.selBest(pop, k=1) )+1 ).flatten() )
        
    # Finalmente se devuelven un vector con el mejor valor encontrado hasta cada generación k así como la mejor solución
    return evals,best_sol        

In [9]:
# En esta parte comprobamos la implementación de los programas. 
# Esta celda no debe ser modificada

popsize = 100
n_gen = 20

# Evaluación del algoritmo para 2 instancias de tamaño 10
filename_base = '../Instances/QAP/Cebe.qap.n10.'
for i in range(2):
  fname  = filename_base+str(i+1)
  evals,best_sol = QAPAdvEA(fname,popsize,n_gen)    
  print(10,i+1,evals,best_sol)  

# Evaluación del algoritmo para 2 instancias de tamaño 50
filename_base = '../Instances/QAP/Cebe.qap.n50.'
for i in range(2):
  fname  = filename_base+str(i+1)
  evals,best_sol = QAPAdvEA(fname,popsize,n_gen)    
  print(50,i+1,evals,best_sol)  


10 1 [7963168.0, 7963168.0, 7718443.0, 7045508.0, 7045508.0, 6350478.0, 5657344.0, 5634884.0, 5564366.0, 5564366.0, 5564366.0, 5564366.0, 5564366.0, 5564366.0, 5564366.0, 5564366.0, 5562141.0, 5562141.0, 5493629.0, 5493629.0] [6, 3, 10, 4, 9, 7, 1, 2, 5, 8]
10 2 [202398.0, 202398.0, 202398.0, 202398.0, 202398.0, 202398.0, 202398.0, 201232.0, 201133.0, 201133.0, 199511.0, 199511.0, 199511.0, 199511.0, 199511.0, 199511.0, 199511.0, 199511.0, 199511.0, 199511.0] [3, 7, 6, 2, 4, 5, 8, 10, 1, 9]


/Users/froux/opt/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/froux/opt/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


50 1 [509048973.0, 509048973.0, 509048973.0, 498435307.0, 498435307.0, 498435307.0, 498435307.0, 498435307.0, 498435307.0, 498435307.0, 498435307.0, 496976171.0, 496976171.0, 496976171.0, 496976171.0, 489621159.0, 489621159.0, 474393392.0, 473667944.0, 473667944.0] [50, 8, 7, 25, 38, 23, 20, 47, 43, 17, 12, 26, 49, 40, 37, 18, 44, 45, 16, 13, 9, 48, 27, 11, 32, 24, 39, 28, 35, 46, 33, 36, 41, 4, 2, 15, 21, 14, 5, 31, 6, 29, 10, 34, 42, 1, 22, 19, 3, 30]
50 2 [5799809.0, 5779989.0, 5749379.0, 5749379.0, 5739463.0, 5726638.0, 5713233.0, 5710248.0, 5710248.0, 5710248.0, 5699448.0, 5697062.0, 5671792.0, 5671792.0, 5671792.0, 5671792.0, 5671792.0, 5671792.0, 5671792.0, 5671792.0] [4, 8, 33, 3, 10, 14, 16, 35, 36, 37, 28, 45, 7, 34, 42, 18, 17, 23, 20, 41, 47, 15, 12, 44, 21, 19, 22, 39, 27, 48, 9, 31, 43, 6, 11, 26, 24, 2, 30, 46, 40, 1, 38, 29, 5, 32, 50, 25, 49, 13]


In [10]:
import numpy as np


def ReadBipPartInstance(fname):
    # Este programa recibe el nombre y ubicación de un fichero con los datos de una instancia del problema BiPartioning
    # y devuelve una estructura con los datos de la instancia.
    hdl = open(fname, 'r')           
    mylist = hdl.readlines()
    hdl.close()
    n = eval(mylist[0])      
    edge_weights = np.zeros((n,n))      # Pesos de las aristas 
    for i in range(n):
        for j,val in enumerate(mylist[i+1].split()):     
            edge_weights[i,j]=eval(val) 
    
    BipPartInstance = {"edge_weights":edge_weights}

    return BipPartInstance

In [11]:
def BipEvaluator(edge_weights,solution):
    n = edge_weights.shape[0]         # Número de nodos
    balance =  np.sum(solution) # Numero de nodos en una de las partes
    fval = 0                  # Peso de las aristas entre partes del grafo
    for i in range(n-1):
        for j in range(i+1,n):
            if solution[i]==1-solution[j]:      # Si estan en partes diferentes  
                fval = fval+edge_weights[i,j]  

    feasible=(balance==n/2)            
    # Finalmente se devuelve el resultado
    return feasible,fval,balance

In [12]:
def SwapAll1AND0(perm):
    n = perm.shape[0]
    ix1 = np.where( perm == 1)[0]
    ix0 = np.where( perm == 0)[0]
    n_neighbors = len(ix1)*len(ix0)  # Número de vecinos    
    neighbors = np.zeros((n_neighbors,n),dtype=int) # Guardaremos todos los vecinos en neighbors
    ind = 0
    for i in range(len(ix1)):
        for j in range(len(ix0)):
            neighbors[ind,:] = perm   
            aux = perm.copy() 
            neighbors[ind,ix1[i]] = aux[ix0[j]]   # Se invierte el camino entre posiciones elegidas
            neighbors[ind,ix0[j]] = aux[ix1[i]]   # Se invierte el camino entre posiciones elegidas
            ind = ind + 1   
    return neighbors

In [13]:
SwapAll1AND0(np.asarray([1,1,0,0]))

array([[0, 1, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 0, 1]])

In [14]:
def BipPartLocalSearch( edge_weights, solution, maxEvals, k ):
    best_vals = []
    best_val = []
    init_sol = solution
    
    # En los siguientes pasos se implementa la búsqueda local para el problema BipPart 
    n = edge_weights.shape[1]   
    feasible,fval,balance = BipEvaluator( edge_weights, init_sol )              # Mejor valor
    best_val = fval
    best_sol = init_sol                                             # Mejor solución 
    improvement = True
    number_evaluations = 0   
    while improvement:                    # Mientras se mejore el valor de la función
        neighbors = SwapAll1AND0(best_sol)       # Todos los vecinos
        n_neighbors = neighbors.shape[0]
        best_val_among_neighbors = best_val
        for i in range(n_neighbors):                    # Se recorren todos los vecinos buscando el mejor 
            sol = neighbors[i,:]  
            feasible,fval,balance =  BipEvaluator( edge_weights, sol )    # Se evalua la función
            if (feasible):
                if fval>best_val_among_neighbors:             # Si es mejor que el mejor valor entre los vecinos hasta el momento
                    best_val_among_neighbors = fval             # se actualiza el mejor valor
                    best_sol_among_neighbors = sol   
            number_evaluations =  number_evaluations + 1  # Se calcula es número de evaluaciones
            if np.mod(number_evaluations,k)==0:
                best_vals.append(best_val_among_neighbors)
        improvement = (best_val_among_neighbors>best_val) #  Se determina si ha habido mejora con respecto al ciclo anterior
        if improvement:                                
            best_val = best_val_among_neighbors           # Se actualiza el mejor valor y la mejor solución 
            best_sol = best_sol_among_neighbors      
            #print(best_val,best_sol, number_evaluations)
        if (number_evaluations > maxEvals):
            break
    # Finalmente se devuelven el mejor valor encontrado así como la mejor solución
    return best_vals,best_sol    

In [15]:
def GRASP_Bipart( solution, edge_weights, max_evals, k):    
    
    n = edge_weights.shape[1] 
    max_dist = np.min(np.min(edge_weights))   # Máxima distancia en la matrix de distancias   
    sol = np.zeros(n,dtype=int)                        # La solución que se construirá      
    cities_to_add = set(range(n))            # Conjunto de ciudades que se adicionarán      
    first_city = np.random.randint(n,size=1)[0]  # La primera ciudad se elige aleatoriamente
    sol[0] = first_city                  
    cities_to_add.remove(first_city)         # Se actualizan los conjuntos de ciudades elegidas y por elegir
    cities_added = set([first_city])   
    last_city = first_city                   # última ciudad añadida hasta el momento
   
    while len(cities_to_add)>0:              # Mientras queden ciudades por añadir
        candidates = np.zeros(len(cities_to_add),dtype=int)         # Se crea una lista de candidatas     
        vect_distances = np.zeros(len(cities_to_add))     # Se crea un vector de probabilidades para cada candidata                                                      
        for i, member in enumerate(cities_to_add):      
            vect_distances[i] = (edge_weights[last_city,member]-max_dist) # Las probabilidades son inv. prop. a las distancias
            candidates[i] = member          
        vect_distances = vect_distances + 1e-6 
        vect_distances = ((vect_distances)/np.sum(vect_distances))  # se normaliza el vector de probabilidades        
        last_city = np.random.choice(candidates,size=1,replace=True,p=vect_distances)[0]  # Se actualiza última ciudad
        sol[len(cities_added)] = last_city                         # Se actualiza la solución y los conjuntos de ciudades
        cities_added.add(last_city)
        cities_to_add.remove(last_city)  # FINAL DEL PROCESO CONSTRUCTIVO
    
    # SE LLAMA AL OPTIMIZADOR LOCAL           
    best_val, best_sol = BipPartLocalSearch( edge_weights, solution[sol], max_evals, k )            
    return best_val, best_sol

In [16]:
# Ejemplo de la implementación de la búsqueda local avanzada para el problema BiPart en Python

def BipPartAdvLocalSearch(fname,solution,max_evals,k):
    
    best_vals = []
    best_sol = solution

    my_BiPart_Instance = ReadBipPartInstance(fname)  # Se lee la instancia
    edge_weights       = my_BiPart_Instance["edge_weights"]
    
    # En los siguientes pasos se implementa la búsqueda local para el problema BiPart 
    best_vals,best_sol = GRASP_Bipart(solution,edge_weights,max_evals,k)
    
    # Finalmente se devuelven el mejor valor encontrado así como la mejor solución
    return best_vals,best_sol        


In [17]:
# En esta parte comprobamos la implementación de los programas. 
# Esta celda no debe ser modificada


maxevals=1000
k=20



# Evaluación del algoritmo para 5 instancias de tamaño n = 10
filename_base = '../Instances/BIPART/Cebe.bip.n10.'
my_solution_10 = np.hstack((np.ones((5)),np.zeros((5))))
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = BipPartAdvLocalSearch(fname,my_solution_10,maxevals,k)  
  print(10,i+1,best_val,best_sol)  



# Evaluación del algoritmo para 5 instancias de tamaño n = 20
filename_base = '../Instances/BIPART/Cebe.bip.n20.'
my_solution_20 = np.hstack((np.ones((10)),np.zeros((10))))
for i in range(5):
  fname  = filename_base+str(i+1) 
  best_val,best_sol = BipPartAdvLocalSearch(fname,my_solution_20,maxevals,k)  
  print(20,i+1,best_val,best_sol)  


# Evaluación del algoritmo para 5 instancias de tamaño n = 50
filename_base = '../Instances/BIPART/Cebe.bip.n50.'
my_solution_50 = np.hstack((np.ones((25)),np.zeros((25))))
for i in range(5):
  fname  = filename_base+str(i+1) 
  best_val,best_sol = BipPartAdvLocalSearch(fname,my_solution_50,maxevals,k)  
  print(50,i+1,best_val,best_sol)  


# Evaluación del algoritmo para 5 instancias de tamaño n = 80
filename_base = '../Instances/BIPART/Cebe.bip.n80.'
my_solution_80 = np.hstack((np.ones((40)),np.zeros((40))))
for i in range(5):
  fname  = filename_base+str(i+1) 
  best_val,best_sol = BipPartAdvLocalSearch(fname,my_solution_80,maxevals,k)  
  print(80,i+1,best_val,best_sol)  




10 1 [777.0, 822.0, 822.0] [0 0 1 0 1 1 0 1 0 1]
10 2 [138874.0, 148084.0, 148084.0] [1 1 0 1 0 0 0 1 0 1]
10 3 [147939.0, 153139.0, 158853.0, 158853.0, 158853.0] [1 0 0 0 1 1 0 1 0 1]
10 4 [166743.0, 174866.0, 174866.0, 180771.0, 180771.0] [1 1 1 0 0 0 0 1 0 1]
10 5 [2428.0, 2484.0, 2484.0, 2530.0, 2530.0, 2530.0] [0 0 1 0 0 1 1 0 1 1]
20 1 [3436.0, 3465.0, 3465.0, 3473.0, 3473.0, 3569.0, 3596.0, 3596.0, 3604.0, 3605.0, 3645.0, 3665.0, 3665.0, 3684.0, 3684.0, 3684.0, 3747.0, 3747.0, 3747.0, 3747.0, 3747.0, 3747.0, 3747.0, 3747.0, 3747.0] [1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1]
20 2 [493959.0, 504231.0, 504463.0, 513898.0, 513898.0, 526283.0, 537014.0, 537014.0, 537014.0, 537014.0, 540455.0, 540455.0, 540455.0, 540455.0, 540455.0, 540455.0, 540455.0, 540455.0, 540455.0, 540455.0] [0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0]
20 3 [570995.0, 575276.0, 589923.0, 589923.0, 589923.0, 605292.0, 605292.0, 605292.0, 605292.0, 608731.0, 619590.0, 619590.0, 619590.0, 619590.0, 619590.0, 619590.0,

80 4 [9424956.0, 9424956.0, 9430796.0, 9437100.0, 9480334.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9482371.0, 9495843.0, 9497862.0, 9497862.0, 9497862.0] [0 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 1 0 0
 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1 

In [18]:
def BipEvaluator(edge_weights,solution):
    n = edge_weights.shape[0]         # Número de nodos
    balance =  np.sum(solution) # Numero de nodos en una de las partes
    fval = 0                  # Peso de las aristas entre partes del grafo
    for i in range(n-1):
        for j in range(i+1,n):
            if solution[i]==1-solution[j]:      # Si estan en partes diferentes  
                fval = fval+edge_weights[i,j]  

    feasible=(balance==n/2)            
    # Finalmente se devuelve el resultado
    return (fval,)#feasible,fval,balance

In [19]:
import random
from deap import algorithms, base, creator, tools
import numpy as np

# En los siguientes pasos se implementa el algoritmo evolutivo para el problema QAP
# Se crea una clase FitnessMax para la maximización de funciones
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Se crea una clase individuo asociada a la clase FitnessMin
creator.create("Individual", list, fitness=creator.FitnessMax)
    
def BipAdvEA(fname,psize,ngen):
    
    evals = np.zeros((ngen))
    best_sol = []

    my_BipPart_Instance = ReadBipPartInstance(fname)  # Se lee la instancia
    edge_weights = my_BipPart_Instance["edge_weights"]
    IND_SIZE = int(fname[-4:-2])
    
    # Heredamos las clases y funciones implementadas como parte de DEAP
    toolbox = base.Toolbox()
    
    # Utilizaremos una representación binaria
    n = int(IND_SIZE/2)
    init_sol = np.hstack((np.ones(n),np.zeros(n)))
    toolbox.register("attr_bool", np.random.permutation, init_sol)
    
    # Definimos que nuestros individuos tendrán secuencias de tamaño IND_SIZE  
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_bool)
    
    # Definimos la población a partir de los individuos
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Asociamos como función de aptitud la función QAPEvaluator
    toolbox.register("evaluate", BipEvaluator,edge_weights)
    
    # Nuestro operador de cruzamiento basado en correspondencia parcial
    toolbox.register("mate", tools.cxTwoPoint)
    
    # El operador de mutación con una probabilidad de mutación de 0.05
    toolbox.register("mutate", tools.mutFlipBit,indpb=0.05)
    
    # Usaremos selección por mejor individuo
    toolbox.register("select", tools.selTournament, tournsize=10)
    
    while True:
        # La población tendrá psize individuos
        pop = toolbox.population(n=psize)
        # El algoritmo evolutivo simple utiliza los siguientes parámetros
        # Probabilidad de cruzamiento 0.5
        # Probabilidad de aplicar el operador de mutación 0.2
        # Número de generaciones ngen
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("max", np.max)
        
        pop, logbook = algorithms.eaSimple(pop, toolbox, stats=stats,cxpb=0.5, mutpb=0.05, ngen=ngen, verbose=False)
        evals = logbook.select('max')[1:ngen+1]
        best_sol = tools.selBest(pop, k=1)
        if (len( np.where( np.asarray( best_sol ) == 1)[0] ) == int(IND_SIZE/2)): 
            # Finalmente se devuelven un vector con el mejor valor encontrado hasta cada generación k así como la mejor solución
            return evals,best_sol 
 

In [20]:
# En esta parte comprobamos la implementación de los programas. 
# Esta celda no debe ser modificada

popsize = 100
n_gen = 20
#  EA para 2 instancias de tamaño n=10
filename_base = '../Instances/BIPART/Cebe.bip.n10.'
for i in range(2):
  fname  = filename_base+str(i+1)
  evals,best_sol = BipAdvEA(fname,popsize,n_gen)  
  print(10,i+1,evals,best_sol)  

#  EA para 2 instancias de tamaño n=50
filename_base = '../Instances/BIPART/Cebe.bip.n50.'
for i in range(2):
  fname  = filename_base+str(i+1)
  evals,best_sol = BipAdvEA(fname,popsize,n_gen)  
  print(50,i+1,evals,best_sol)  



10 1 [803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0, 803.0] [[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]]
10 2 [148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0, 148084.0] [[0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]]
50 1 [21964.0, 21964.0, 21964.0, 21964.0, 21969.0, 21969.0, 22110.0, 22110.0, 22110.0, 22351.0, 22351.0, 22390.0, 22390.0, 22390.0, 22390.0, 22390.0, 22390.0, 22390.0, 22390.0, 22470.0] [[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]]
50 2 [3016453.0, 3043892.0, 3043892.0, 3043892.0, 3048780.0, 3048780.0, 3057100.0, 3068999.0, 3068999.0